<center>
    
    Sudoku
    
    Author: Daniel Coble
    
    Status: Work in progress
</center>

Sudoku solving by hand uses some quite [involved logic](https://www.ijcsi.org/papers/IJCSI-11-2-1-247-253.pdf) in recognizing patterns to eliminate possible digits from each position, and eventually determine the solution. Programming all of these patterns would be cumbersome, but much of the work can be done by using two tricks, which I am calling inclusive and exclusive set logic.

A sudoku board consists of 27 sets of 9 elements. Each element is a digit 1-9, which appears exactly once in each set. Exclusive logic is very easy. If we know that a certain digit must exist in the intersection between set $A$ and set $B$, $A\cap B$,then that digit cannot exist in the compliments of those two sets $A-B$ or $B-A$.

Inclusive logic, called twins, triplets, etc. in sudoku, is a bit harder to understand. If we have a set $A$, and a subset of elements $B\subset A$, and there exists a subset of digits $D$  such that all digits of $D$ in $A$ are also in $B$, and $||B||=||D||$, no digits outside $D$ may exist in $B$.

In this notebook, I create an object oriented way to solve sudoku using the logic described above. Sudoku is represented as a specific instance of a graph coloring problem with multiedges. The solver repeatedly checks for exclusive logic, then inclusive logic. Finally, after if it cannot detect any more logic, it attempts soft brute forcing by assuming and digit and searching for a contradiction. Soft brute forcing is a recursive procedure.

In [ ]:
import numpy as np
class Cell():
    '''
    notes: np array of bools, True if it is deemed possible for that number to be in the cell
    '''
    def __init__(notes):
        self.notes = notes
        self.solved = np.sum(notes) == 1



class Sudoku:
    
    def __init__(board):
        
        self.notes = np.zeros((9,9,9), dtype=bool)
        